# Principal Components Analysis: Dunkirk XRF Data

Code snippets showing the basic steps used to build the Dunkirk PCA dashboard including:
* Data import
* Visual data inspection via box plots by element and element/lithology
* Application of PCA using NumPy
* Application of PCA using sklearn
* PCA interpretation via scatter plots and bar plots

## Data Import
For this example, the data is stored in csv files, but the web-based dashboard uses a postgreSQL database for all data.  

The ```data``` file includes the corrected XRD elemental data with all elemental measurements reported in ppm.  The ```meta``` file contains location and lithology specific information about each sample.  The 'data' and the 'meta' file are merged into a data frame (df) that will be used for subsequent analytics.    

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import plotly.graph_objects as go
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

# imports
data = pd.read_csv('data_chemostrat_benchtop_compiled.csv')
meta = pd.read_csv('info_chemostrat_benchtop_compiled.csv')
data.drop(columns=['Sample ID'], inplace=True) # drop a repeat column before merge
# merge data and metadata
df = data.merge(meta, on='key')
# check the head of the df
df.head()

## Visually inspect the data (box plots)
The modules below will be used to define an element list and generate a box plot and lithology specific plots for visual inspection of the data. The box plot is used to identify unusual data points that warrant further investigation.  It is also helpful for gaining a preliminary understanding of how the data are distributed. 

The code block below contains 3 primary components:
* **Component 1**: Definition of 3 lists that are used for both plotting and dropdown menus
* **Component 2**: Definition of the application (```app```) layout.  The layout defined in this Jupyter notebook a slightly more streamlined than that of the actual dashboard as only one ```div``` is used.
* **Component 3**: Definition of ```callback``` function allowing for updating the box plots according to the user input. 

The above components, specifically component 2 and 3 will be repeated in subsequent code blocks but will not be called out as was done in the code block below.

In [ ]:
# --------- COMPONENT 1 ---------
elem = [column for column in df.columns if len(column)<=2] # list of elements
yvals=[300000, 100000, 10000, 1000, 100, 50] # y-axis values in ppm that are used to rescale the range on the box plot
ylab = ['300K ppm', '100K ppm', '10K ppm', '1K ppm', '100 ppm', '50 ppm'] # y-axis labels that the user will see

# --------- COMPONENT 2 ---------
app = JupyterDash(__name__) # initiate the app

# define layout containing a dropdown menu and a box plot
app.layout = html.Div([
    html.Div([
        html.H3('Select a y-axis max value:'),
        dcc.Dropdown(
        id='yaxis_dropdown',
        options=[{'label': ylab[i], 'value': yvals[i]}
                for i in range(len(yvals))
                ],
        value=300000,
        multi=False
                )
        ], style={'width':'20%'}),  
    # box plots for elements and elements by lithology (defined in callback function)
    dcc.Graph(id='box')
])

# --------- COMPONENT 3 ---------
# callback function that updates the box plot to honor changes to the y-axis range. 
@app.callback(
    Output('box', 'figure'),
    [Input('yaxis_dropdown', 'value')])
def update_BoxWhisker(value): 
    # the metrics are every parameter that the user has selected 
    boxplot = {'data':[
                        go.Box(y=df[e], name=e, boxmean='sd',
                               boxpoints='outliers',
                               # outliers are defined as 1.5 +/- IQR per plotly 
                               marker={'color':'lightseagreen'},
                               text=df['Lithology']) for e in elem],
                            
                            'layout':go.Layout(title='Elemental Data Box Plots',
                                               xaxis={'title':'Elements'},
                                               yaxis={'title':'ppm', 'type':'linear',
                                                      'range':[0,value]},
                                               annotations=[dict(x=20, y=value*0.8, 
                                                                 text='Sample Count = ' + str(len(df)), 
                                                                 showarrow=False)])
                }
    return boxplot

app.run_server(mode='inline', port=8050)

### Box plots grouped by lithology
The approach below is like that shown above, but now the differences in elemental distributions as a function of lithology can be observed.  This visualization is helpful for understanding the some of the data points that appear to be unusual but are in fact a reflection of lithologic changes. 

Note that ```app``` is defined again in this module, but this is not the case in the web-based dashboard.  Redefining ```app``` and ```app.run_server``` are necessary for the Jupyter notebook examples only.  

In [ ]:
app = JupyterDash(__name__) # initiate the app
# define the layout
app.layout = html.Div([
                html.Div([
                    html.H5('Select an Element:'),
                    dcc.Dropdown(
                        id='element_dropdown',
                        options=[{'label':e, 'value':e} for e in elem],
                        value='Al',
                        multi=False
                                )
                        ], style={'width':'20%'}),
                    dcc.Graph(id='box-lith')

                    ])

# define the callback function
@app.callback(
    Output('box-lith', 'figure'),
    [Input('element_dropdown', 'value')])
def update_BoxWhiskerLith(value): 
    
    # the metrics are every parameter that the user has selected 
    boxLith_plot = {'data':[
                        go.Box(y=df[df['Lithology']=='Grey Shale'][value], 
                               boxmean='sd',
                               boxpoints='outliers',
                               # outliers are defined as 1.5 +/- IQR per plotly 
                               marker={'color':'grey'},
                               name='Grey Shale' ),
                        
                        go.Box(y=df[df['Lithology']=='Black Shale'][value], 
                               boxmean='sd',
                               boxpoints='outliers',
                               # outliers are defined as 1.5 +/- IQR per plotly 
                               marker={'color':'black'},
                               name='Black Shale' ),
                        
                        go.Box(y=df[df['Lithology']=='Black Shale/pyrite'][value], 
                               boxmean='sd',
                               boxpoints='outliers',
                               # outliers are defined as 1.5 +/- IQR per plotly 
                               marker={'color':'gold'},
                               name='Black Shale/Pyrite' )],
                            
                            'layout':go.Layout(title='Elemental Data Box Plots Grouped by Lithology',
                                               xaxis={'title':str(value)},
                                               yaxis={'title':'ppm', 'type':'linear'})
                                               }
    return boxLith_plot

app.run_server(mode='inline', port=8060)

## Principal Components Analysis (PCA)

For illustrative purposes, two methods for applying PCA have been included below.  The first method uses _NumPy_ only to implement PCA and the second uses _sklearn_ for a more direct application.  Both methods are applied to a toy data set followed by application of the sklearn method to the standardized XRF data.  

The basic steps for applying PCA are as follows:
1. Review data and identify any data that may warrant further investigation
2. Standardize or normalize the data.
    * **Standardize**: mean of zero and standard deviation of 1 (standardization was used here)
    * **Normalize**: scale each input to have max value 1 and min of 0 (or some constant range)
3. Calculate covariance matrix
4. Extract eigenvalues and eigenvectors from covariance matrix
    * **eigenvalues**: magnitude of the eigenvector and explain the amount of variance explained by a given eigenvector.  The higher the eigenvalue the more of the overall variance that is captured the respective eigenvector.  
    * **eigenvector**: direction of variance as defined by contributions from each variable.  The eigenvectors are the principal components.  
5. Rank the principal components from most to least variance explained.  This is done by using the eigenvalues, where the highest eigenvalue corresponds to PC1, the second highest to PC2, and so on. 
6. Determine the number of principal components to retain, which will be project specific.  Typically, the number of components necessary to account for >80% of the overall variance is used as a rule of thumb. 

When using _sklearn_, steps 3, 4, and 5 are handled in the background by the PCA class.  

### NumPy PCA, Toy Dataset (this is an aside to the XRF dataset)

To illustrate the steps listed above, a toy dataset will be used.  The toy dataset is taken from **_Statistics and Data Analysis in Geology, Davis, pg. 509_**.  The _NumPy_ output will be compared to the _sklearn_ output.  

**Note**: The standardization step is skipped in this example because this dataset contains only two variables with values in the same units of measurement.  

In [ ]:
x1 = [3,4,6,6,6,7,7,8,9,9,9,10,11,12,12,13,13,13,13,14,15,17,17,18,20] # variable 1
x2 = [2,10,5,8,10,2,13,9,5,8,14,7,12,10,11,6,14,15,17,7,13,13,17,19,20] # variable 2
x = np.array([x1,x2]) # convert x1 and x2 into a matrix
x

Plot the data

In [ ]:
fig = go.Figure() # initiate a figure
# add the scatter plot
fig.add_trace(go.Scatter(
    x=x[0,:], y=x[1,:],
    mode='markers')
    )
# add chart and axis titles
fig.update_layout(title='x1 vs x2 Scatter Plot',
                 xaxis={'title':'x1'},
                 yaxis={'title':'x2'},
                 autosize=False,
                 width=750,
                 height=750)

fig.show()

Generate the covariance matrix from x.

In [ ]:
cov_matrix = np.cov(x) # covariance matrix
cov_matrix

Extract the eigenvalue (w) and the eigenvector (v) from the covariance matrix (s).

In [ ]:
numpy_eigenvalues,numpy_eigenvectors = np.linalg.eig(cov_matrix)
print('Eigenvectors: \n', numpy_eigenvectors, '\n')
print('Eigenvalues: \n', numpy_eigenvalues)

Sort the **eigenvectors** from greatest to least **eigenvalue**

In [ ]:
temp = np.vstack((numpy_eigenvectors,numpy_eigenvalues)).T # use a temp array to stitch the eigenvectors and eigenvalues together
temp_sort = temp[np.argsort(-temp[:, 2])] # sort the array by eigenvalues descending order
numpy_eigenvectors_sort = temp_sort[:,0:2] # sorted eigenvectors by descending eigenvalue
numpy_eigenvalues_sort = temp_sort[:,2] # sorted eigenvalues, descending order

print('Sorted Eigenvectors: \n', numpy_eigenvectors_sort, '\n', 'NOTE: each column is an eigenvector', '\n')
print('Sorted Eigenvalues: ', '\n', numpy_eigenvalues_sort)

In [ ]:
# starting point of eigenvectors 1 and 2 (i.e., PC1 and PC2) for illustrative purposes
evec1 = numpy_eigenvectors_sort[:,0] * numpy_eigenvalues_sort[0]*-1 # coordinates of eigenvector 1 (PC1)
evec2 = numpy_eigenvectors_sort[:,1] * numpy_eigenvalues_sort[1]*-1 # coordinates of eigenvector 2 (PC2)
# multiplication by -1 is for plotting convinience only
evec1 = np.vstack(([0,0], evec1))
evec2 = np.vstack(([0,0], evec2))
print(evec1)

fig = go.Figure() # initiate a figure
# add the scatter plot
fig.add_trace(go.Scatter(
    x=x[0,:] , y=x[1,:],
    mode='markers',
    name='data')
    )
fig.add_trace(go.Scatter( # multiply x an y by 0.5 so half the variance is in in quad 1 and the other half in quad 3
    x=evec1[:,0]*0.5 + np.mean(x[0,:]), y=evec1[:,1]*0.5 + np.mean(x[1,:]), name='PC1_quadrant1', mode='lines', marker={'color':'red'}
    ))
fig.add_trace(go.Scatter(
    x=evec1[:,0]*(-0.5) + np.mean(x[0,:]), y=evec1[:,1]*(-0.5) + np.mean(x[1,:]), name='PC1_quadrant3', mode='lines', marker={'color':'red'},
    line = dict(shape = 'linear', color = 'red', width= 2, dash = 'dash')
    ))

fig.add_trace(go.Scatter( # multiply x and y by 0.5 so half the variance is in quad 2 and the other half in quad 4
    x=evec2[:,0]*(-0.5) + np.mean(x[0,:]), y=evec2[:,1]*(-0.5) + np.mean(x[1,:]), name='PC2_quadrant2', mode='lines', marker={'color':'black'}
    ))
fig.add_trace(go.Scatter(
    x=evec2[:,0]*0.5 + np.mean(x[0,:]), y=evec2[:,1]*0.5 + np.mean(x[1,:]), name='PC2_quadrant4', mode='lines', marker={'color':'black'},
    line = dict(shape = 'linear', color = 'black', width= 2, dash = 'dash')
    ))
# add chart and axis titles
fig.update_layout(title='x1 vs x2 Scatter Plot with PC1 and PC2 Overlay',
                 xaxis={'title':'x1', 'range':[-5,25]},
                 yaxis={'title':'x2', 'range':[-5,25]},
                 autosize=False,
                 width=750, # use a fixed width and height so that PC1 and PC2 plot orthogonal to each other
                 height=750)

fig.show()

Amount of variance explained by each component

In [ ]:
numpy_total_variance = np.sum(numpy_eigenvalues_sort)
numpy_pct_variance = numpy_eigenvalues_sort / numpy_total_variance * 100
numpy_pct_variance

**PCA Score and Correlation**

The PCA score is the result of the matrix multiplication of the input data with the eigenvectors.  PCA scores for each sample are calculated. The score shows a given sample's affinity for each principal component. For example, sample-1 will have a score for principal component 1 (PC_1) and principal component 2 (PC_2). A sample (row of data) that scores high for PC_1 indicates that this sample was enriched in elements that account for most of the variance in this component.  Samples with a low negative score for a given PC indicate that they vary inversely with the same PC.  

After the PCA scores are calculated each score is correlated against the input data (```xt``` in this case).  The trend of the correlation will be identical to that of the eigenvectors (when the data is standardized). The correlation comparison helps with interpretation of each principal component (eigenvector).

In [ ]:
xt = x.T # transpose of input data for proper alignment in matrix multiplication
numpy_pcaScore = np.matmul(xt, numpy_eigenvectors_sort) # matrix multiplication
# correlation between input data, xt, and the PCA score
numpy_pcaCorr = np.corrcoef(xt, numpy_pcaScore, rowvar=False) # correlation matrix
numpy_pcaCorr = numpy_pcaCorr[0:len(numpy_eigenvectors_sort), len(numpy_eigenvectors_sort)::]*-1
# convert to a dataframe for easier viewing
numpy_pcaCorr_df = pd.DataFrame(numpy_pcaCorr, index=['x1', 'x2'], columns=['PC1', 'PC2'])
numpy_pcaCorr_df

**Bar Plots of eigenvectors and PCA score correlation**

These plots are helpful in interpretation of the pca output.

In [ ]:
# --------- EIGENVECTOR WEIGHTS BY VARIABLE X1, X2  ---------
fig = go.Figure()
fig.add_trace(go.Bar(x=['x1', 'x2'], y=numpy_eigenvectors_sort[:,0]*(-1),
           marker={'color':'red'},
           name='PC1'))
fig.add_trace(go.Bar(x=['x1', 'x2'], y=numpy_eigenvectors_sort[:,1]*(-1),
           marker={'color':'grey'},
           name='PC2'))

# update the layout
fig.update_layout(title='Eigenvector Weights by Variable',
                   xaxis={'title':'Variables'},
                   yaxis={'title':'Eigenvector weight (normalized variance)'})
fig.show()

# --------- CORRELATION OF PCA SCORE VS VARIABLES X1, X2 ---------
fig = go.Figure()
fig.add_trace(go.Bar(x=['x1', 'x2'], y=numpy_pcaCorr[:,0],
           marker={'color':'red'},
           name='PC1'))
fig.add_trace(go.Bar(x=['x1', 'x2'], y=numpy_pcaCorr[:,1],
           marker={'color':'grey'},
           name='PC2'))

# update the layout
fig.update_layout(title='Correlation of PCA Score vs Variables',
                   xaxis={'title':'Variables'},
                   yaxis={'title':'Correlation Coefficient'})
fig.show()

### Sklearn PCA, Toy Dataset (one more time)
This example is identical to the _NumPy_ example above, except it will be much more compact since _sklearn_ is being used.  The print statements in the block below show direct comparisons of both the _NumPy_ and _sklearn_ instances of the PCA applied to the toy dataset. It is evident from the max difference comparisons that both methods are effectively identical.  Using _sklearn_ is a more compact method for applying PCA, but it is helpful to use the _NumPy_ method for better understanding of the PCA workflow.  

In [ ]:
x1 = [3,4,6,6,6,7,7,8,9,9,9,10,11,12,12,13,13,13,13,14,15,17,17,18,20] # variable 1 (copy of data from NumPy example)
x2 = [2,10,5,8,10,2,13,9,5,8,14,7,12,10,11,6,14,15,17,7,13,13,17,19,20] # variable 2 (copy of data from NumPy example)
x = np.array([x1,x2]).T # convert x1 and x2 into a matrix (note that x has to be transposed prior to sklearn PCA)

# the two lines below are the entire pca using sklearn
pca = PCA(n_components=2) # instantiate a PCA
sklearn_pca = pca.fit(x) # fit the PCA to the data (x)

sklearn_eigenvectors = sklearn_pca.components_ # eigenvector attribute of sklearn_pca
sklearn_eigenvalues = sklearn_pca.explained_variance_ # eigenvalue attribute of sklearn_pca
sklearn_pctVar = sklearn_pca.explained_variance_ratio_ # percent of explained variance attribute of sklearn_pca

# print output to compare sklearn and NumPy)
print('Eigenvectors from sklearn: \n', sklearn_eigenvectors, '\n')
print('Eigenvectors from NumPy: \n', numpy_eigenvectors_sort*-1, '\n')
print('Eigenvalues from sklearn: \n', sklearn_eigenvalues, '\n')
print('Eigenvalues from NumPy: \n', numpy_eigenvalues_sort, '\n')
print('Percent variance from sklearn: ', sklearn_pctVar *100)
print('Percent variance from NumPy: ', numpy_pct_variance, '\n')
print('Eigenvector max difference (sklearn vs NumPy): ', np.max(sklearn_eigenvectors - numpy_eigenvectors_sort*(-1)))
print('Eigenvalue max difference (sklearn vs NumPy): ', np.max(sklearn_eigenvalues - numpy_eigenvalues_sort))
print('Percent variance max difference (sklearn vs NumPy): ', np.max(sklearn_pctVar*100 - numpy_pct_variance))


## PCA applied to XRF Data
In this section a PCA will be applied to standardized XRF data using _sklearn_. Not shown here, but the PCA using the non-standardized data is very helpful for understanding the elements that are most influential. 

**Step 1** Standardize the XRF data

In [ ]:
# data_stdz will be a numpy array containing the standardized XRF data.  
dataStdz = StandardScaler().fit_transform(df[elem])

**Step 2** Use _sklearn_ PCA to define the PCA model

In [ ]:
# the number of components will be equal to the number of elements, but reduced in later steps
pca_model = PCA(n_components=len(elem)) # 42 components (only 10 will be used)
pca = pca_model.fit(dataStdz)                       # fit the model
eigenvalues_dataStdz = pca.explained_variance_      # eigenvalues (magnitude of each principal component)
eigenvectors_dataStdz = pca.components_             # eigenvectors (principal component vectorrs)
explainVar_dataStdz = pca.explained_variance_ratio_ # fraction of variance explained by each principal component

### Plot the PCA Output

A total of 4 plots will be used to gain understanding of the PCA output:

* **Plot 1**: Bar plot showing the variance contribution of each PC
* **Plot 2**: 3D scatter plot showing the PC-scores colored by lithology
* **Plot 3**: Bar plot showing each elements contribution to the variance of a given PC
* **Plot 4**: Bar plot showing the correlation of each elements PC-score with the input elemental data 

**Plot 1**: Bar plot showing the variance contribution of each PC

In [ ]:
# list of column names for the bar chart
pc_cols = ['pc_'+str(i+1) for i in range(len(eigenvectors_dataStdz))]
# cumulative contribution to the variance
cumVar_dataStdz = np.cumsum(explainVar_dataStdz)
# generate the figure
fig = go.Figure()
fig.add_trace(go.Bar(x=pc_cols[0:10], y=explainVar_dataStdz[0:10],
           marker={'color':'deeppink'},
           name='Inc. Var.'))
fig.add_trace(go.Bar(x=pc_cols[0:10], y=cumVar_dataStdz[0:10],
           marker={'color':'grey'},
           name='Cum. Var.'))
# update the layout
fig.update_layout(title='Percent Variance (cropped to first 10 components)',
                   xaxis={'title':'Principal Components'},
                   yaxis={'title':'Fraction of Variance Accounted For'})
fig.show()


**Plot 2**: 3D scatter plot showing the PC-scores colored by lithology

Visualizing more than 3 dimensions is difficult, but the plot below allows for visualization of any combination of 3 principal components for PC 1-10.  It is evident from the plot that the first 3 PCs effectively differentiate the 3 primary lithologies described thus far (Black Shale/Pyrite, Black Shale, and Grey Shale). The lithology labels are based on field descriptions for each section for which samples were collected.  When viewing PC_1, PC_2, and PC_3 it is evident that the Black Shale/Pyrite lithology is characterized by elevated PC_1 scores, the Black Shale lithology by elevated PC_2 scores, and the Grey Shale lithology has the highest (and lowest) PC_3 scores, but is also low in PC_2 and PC_1 scores.

In [ ]:
# calculate the PCA scores for the standardized xrf data
dataStdz_scores = np.matmul(dataStdz, eigenvectors_dataStdz.T)
# convert scores to dataframe
dataStdz_scores_df = pd.DataFrame(dataStdz_scores, columns=pc_cols)
# add lithology labels
dataStdz_scores_df['lith'] = list(df['Lithology'])

app = JupyterDash(__name__) # initiate the app

app.layout = html.Div([
       html.Div([
        html.H5('X-Axis'),
        dcc.Dropdown(
            id='x_3d_stand_pca',
            options=[{'label':'PC_'+str(i+1), 'value':'pc_'+str(i+1)} for i in range(10)],
            value='pc_1',
            multi=False)
       ], style={'width':'20%', 'display':'inline-block'}),
       html.Div([
        html.H5('Y-Axis'),
        dcc.Dropdown(
            id='y_3d_stand_pca',
            options=[{'label':'PC_'+str(i+1), 'value':'pc_'+str(i+1)} for i in range(10)],
            value='pc_2',
            multi=False),
        ], style={'width':'20%', 'display':'inline-block', 'margin-left':'10px'}),
        html.Div([
        html.H5('Z-Axis'),
        dcc.Dropdown(
            id='z_3d_stand_pca',
            options=[{'label':'PC_'+str(i+1), 'value':'pc_'+str(i+1)} for i in range(10)],
            value='pc_3',
            multi=False),
       ], style={'width':'20%', 'display':'inline-block','margin-left':'10px'}),
    
        dcc.Graph(id='scatter3d')
        ])
@app.callback(
    Output('scatter3d', 'figure'),
    [Input('x_3d_stand_pca', 'value'),
     Input('y_3d_stand_pca', 'value'),
     Input('z_3d_stand_pca', 'value')]
    )
def update_3dScatter_pca(x_value, y_value, z_value):
    
    pca_scatter = {'data':[go.Scatter3d(
                    x=dataStdz_scores_df[x_value][dataStdz_scores_df['lith']=='Grey Shale'],
                    y=dataStdz_scores_df[y_value][dataStdz_scores_df['lith']=='Grey Shale'],
                    z=dataStdz_scores_df[z_value][dataStdz_scores_df['lith']=='Grey Shale'],
                    mode='markers',
                    marker = {
                        'size':8,
                        'line':{'color':'white', 'width':3},
                        'opacity':0.6,
                        'color':'grey',
                        },
                    name='Grey Shale'),
                    go.Scatter3d(
                    x=dataStdz_scores_df[x_value][dataStdz_scores_df['lith']=='Black Shale'],
                    y=dataStdz_scores_df[y_value][dataStdz_scores_df['lith']=='Black Shale'],
                    z=dataStdz_scores_df[z_value][dataStdz_scores_df['lith']=='Black Shale'],
                    mode='markers',
                    marker = {
                        'size':8,
                        'line':{'color':'white', 'width':3},
                        'opacity':0.6,
                        'color':'black',
                        },
                    name='Black Shale'),
                    go.Scatter3d(
                    x=dataStdz_scores_df[x_value][dataStdz_scores_df['lith']=='Black Shale/pyrite'],
                    y=dataStdz_scores_df[y_value][dataStdz_scores_df['lith']=='Black Shale/pyrite'],
                    z=dataStdz_scores_df[z_value][dataStdz_scores_df['lith']=='Black Shale/pyrite'],
                    mode='markers',
                    marker = {
                        'size':8,
                        'line':{'color':'white', 'width':3},
                        'opacity':0.6,
                        'color':'gold',
                        },
                    name='Black Shale/Pyrite')],
                    'layout':go.Layout(title='3D Scatterplot - PC Score (standardized data)',
                                   height=500,
                                   width=600,
                                   scene={'xaxis':{'title':'X-'+x_value},
                                          'yaxis':{'title':'Y-'+y_value},
                                          'zaxis':{'title':'Z-'+z_value}})
                        }
    
    return pca_scatter

app.run_server(mode='inline', port=8080)

**Plot 3 and Plot 4 Shown Together**

Plot 3 --> Bar plots showing each elements contribution to the variance of a given PC.  
Plot 4 --> Bar plot showing the correlation of each elements PC-score with the input elemental data

In [ ]:
# eigenvectors (principal components) in df format with labeled indices and columns
eigenvectors_dataStdz_df = pd.DataFrame(eigenvectors_dataStdz, index=pc_cols, columns=elem)
# determine the correlation between the pca scores and the standardized data
dataStdz_pca_corr = np.corrcoef(dataStdz, dataStdz_scores, rowvar=False)
# extract the upper right quadrant of the correlation matrix (could also use lower left)
dataStdz_pca_corr = dataStdz_pca_corr[0:len(elem), len(elem)::]
# add index and column labels to the correlation matrix
dataStdz_pca_corr_df = pd.DataFrame(dataStdz_pca_corr, index=elem, columns=pc_cols)

app = JupyterDash(__name__) # initiate the app

app.layout = html.Div([
        html.Div([
                html.H5('Select PC'),
                dcc.Dropdown(
                    id='pc_dd_bar_ppm_stand',
                    options=[{'label':'PC_'+str(i+1), 'value':'pc_'+str(i+1)} for i in range(10)],
                    value='pc_1',
                    multi=False
                    )
                ], style={'width':'20%'}),
        html.Div(dcc.Graph(id='bar-pca-ppm-stand')),
        html.Div(dcc.Graph(id='bar-pca-ppm-corr-stand'))
    ])

@app.callback(
    Output('bar-pca-ppm-stand', 'figure'),
    Output('bar-pca-ppm-corr-stand', 'figure'),
    [Input('pc_dd_bar_ppm_stand', 'value')])
def update_pca_ppm_bar(value): 
    
    bar_pca_ppm ={'data':[
    go.Bar(x=eigenvectors_dataStdz_df.columns, y=eigenvectors_dataStdz_df.loc[value, :],
               marker={'color':'deeppink'})],
    
    'layout':go.Layout(title='Normalized Variance Contribution, ' + value.upper(),
                       xaxis={'title':'Variables (Elements)'},
                       yaxis={'title':'Normalized Variance'})
    }
    
    bar_pca_ppm_corr={'data':[
    go.Bar(x=dataStdz_pca_corr_df.index, y=dataStdz_pca_corr_df[value],
               marker={'color':'deeppink'})],
    
    'layout':go.Layout(title='Correlation of select PC vs Elements, '+ value.upper(),
                       xaxis={'title':'Variables (Elements)'},
                       yaxis={'title':'Correlation Coefficient (-1,+1)'})
                        }
    
    return bar_pca_ppm, bar_pca_ppm_corr

app.run_server(mode='inline', port=8090)

## What does all of this mean?

_Performing the PCA took a number of steps, was it all worth it and what does it help to clarify?_
* The PCA was being used, in part, as a data exploration method.  Specifically, the hope is to improve understanding of how the various elements relate to each other and to the lithologies.
* Interpreting >40 elements is difficult, but doing so using a dimensionality reduction tool like PCA makes this task more manageable.  
* The approach was effective for detecting differences between the three primary lithologies.  It was also effective at highlighting the lithology (Black Shale/Pyrite) that holds the highest potential of containing critical minerals.  

_What's next?_
* This dataset includes only a portion of the data from 3 geographic locations. In total ~20 locations will be sampled and many (hundreds) of additional samples will be available for further PCA.
* PCA is just one of many methods for reducing dimensionality.  Other methods, such as k-means analysis, can be helpful for clustering data and clarifying groups.  At the very least one other method will be used (likely k-means) to identify key groupings of elements.  
* This study is largely opensource, so suggestions are welcome for other data exploration methods.  